# SEGAN 
### Speech Enhancement GAN(Generative Adversarial Network) in TensorFlow
Hyungon Ryu | Sr Solution Architect @ NVIDIA

This jupyter implement  of [SEGAN](https://github.com/santi-pdp/segan) project  in COLAB. The original paper can be found [here](https://arxiv.org/abs/1703.09452) , and test samples are available [here](http://veu.talp.cat/segan/).

![SEGAN model image](https://github.com/santi-pdp/segan/raw/master/assets/segan_g.png)

This model deals with raw speech waveforms on many noise conditions at different SNRs (40 at training time and 20 during test). It also models the speech characteristics from many speakers mixed within the same structure (without any supervision of identities), which makes the generative structure generalizable in the noise and speaker dimensions.

There are two repositories that were good references on how GANs are defined and deployed:

 - [improved-gan](https://github.com/openai/improved-gan): implementing improvements to train GANs in a more stable way
 - [DCGAN-tensorflow](https://github.com/carpedm20/DCGAN-tensorflow): implementation of the DCGAN in tensorflow

### dependency
This segan project requres sox, pysox  and scipy to handle wave files. 

In [1]:
 %%time 
 # for requirement
 !apt-get install sox libsox-dev libsox-fmt-all
 !pip install numpy  scipy   toml sox tqdm

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  dbus libao-common libao4 libapparmor1 libasound2 libasound2-data libasyncns0
  libdbus-1-3 libflac8 libgsm1 libid3tag0 libltdl7 libmad0 libmagic-mgc
  libmagic1 libmp3lame0 libogg0 libopencore-amrnb0 libopencore-amrwb0
  libpulse0 libsndfile1 libsox-fmt-alsa libsox-fmt-ao libsox-fmt-base
  libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse libsox2 libtwolame0
  libvorbis0a libvorbisenc2 libvorbisfile3 libwavpack1 libwrap0 tcpd
Suggested packages:
  default-dbus-session-bus | dbus-session-bus libaudio2 libesd0 | libesd-alsa0
  libasound2-plugins alsa-utils file pulseaudio
The following NEW packages will be installed:
  dbus libao-common libao4 libapparmor1 libasound2 libasound2-data libasyncns0
  libdbus-1-3 libflac8 libgsm1 libid3tag0 libltdl7 libmad0 libmagic-mgc
  libmagic1 libmp3lame0 libogg0 libopencore-amrnb0 libopencore-amrwb0
  libpul

## Dataset
The speech enhancement dataset used can be found in [Edinburgh DataShare](http://datashare.is.ed.ac.uk/handle/10283/1942).  In COLAB environment, you will lose the downloaded dataset in every reconnection. So I recommend to mount google drive in COLAB. for each connection, a time for each connection in COLAB.


There are [GUIDE](https://colab.research.google.com/notebooks/io.ipynb) and example for External data: Drive, Sheets, and Cloud Storage in COLAB

 I assume you already upload below 4 files in google drive 
- clean_testset_wav.zip	
- noisy_testset_wav.zip
- clean_trainset_wav.zip	
- noisy_trainset_wav.zip

instead of using data_prepare.sh, handle data step by step including python script to downsample.

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


I already upload each zip files in ''COLAB/segan' folder in google Drive. and target directory name is 'dataset-segan' in local directory in COLAB.
It will take about 40 seconds to copy.

In [26]:
%%time
!ls -h   "drive/My Drive/COLAB/segan/"
!cp  -rf "drive/My Drive/COLAB/segan" ./dataset-segan/
!ls -h ./dataset-segan


clean_testset_wav.zip	noisy_testset_wav.zip	segan.tfrecords
clean_trainset_wav.zip	noisy_trainset_wav.zip
^C
noisy_trainset_wav.zip	segan.tfrecords
CPU times: user 235 ms, sys: 49.5 ms, total: 285 ms
Wall time: 43.6 s


In [4]:
!ls -h ./dataset-segan
!rm -rf data

clean_testset_wav.zip	noisy_testset_wav.zip
clean_trainset_wav.zip	noisy_trainset_wav.zip


### uncompress dataset
unzip the file in data folder and remove zip files to save storage.  It will take 1 minutes to finish. test dataset have their subfolder so unzip command are different. 

total size of uncompressed data are 2.5GB as below
```
192M	data/noisy_testset_wav
1.1G	data/clean_trainset_wav
192M	data/clean_testset_wav
1.1G	data/noisy_trainset_wav
2.5G	data
```
We  also check the raw files. 
```
p226_001.wav
p226_002.wav
p232_001.wav
p232_002.wav
p226_001.wav
p226_002.wav
p232_001.wav
p232_002.wav
```


In [5]:
%%time
!mkdir data
!unzip -q ./dataset-segan/clean_trainset_wav.zip -d ./data/clean_trainset_wav
!unzip -q ./dataset-segan/clean_testset_wav.zip  -d ./data
!unzip -q ./dataset-segan/noisy_trainset_wav.zip -d ./data/noisy_trainset_wav
!unzip -q ./dataset-segan/noisy_testset_wav.zip  -d ./data
!du -h data
!ls ./data/clean_trainset_wav | head -n 2
!ls ./data/clean_testset_wav  | head -n 2
!ls ./data/noisy_trainset_wav | head -n 2
!ls ./data/noisy_testset_wav  | head -n 2

192M	data/noisy_testset_wav
1.1G	data/clean_trainset_wav
192M	data/clean_testset_wav
1.1G	data/noisy_trainset_wav
2.5G	data
p226_001.wav
p226_002.wav
p232_001.wav
p232_002.wav
p226_001.wav
p226_002.wav
p232_001.wav
p232_002.wav
CPU times: user 411 ms, sys: 88.3 ms, total: 499 ms
Wall time: 1min 1s


### subsampling
In segan implementation use 16Khz sampling rate but the sampling rate of dataset are 48Khz. So we need to downsampling for these dataset  with pysox. 

we will use python script instead of sox in bash script as below :
```Shell
    if [ ! -d clean_trainset_wav_16k ]; then
        echo 'CONVERTING CLEAN WAVS TO 16K...'
        mkdir -p clean_trainset_wav_16k
        pushd clean_trainset_wav
        ls *.wav | while read name; do
            sox $name -r 16k ../clean_trainset_wav_16k/$name
        done
        popd
    fi
    ```


It takes about 6 minuets to finish subsampling

In [6]:
%%time
import os
import sox
import tqdm

tfm = sox.Transformer()
tfm.set_input_format( rate=32000  )
tfm.set_output_format( rate=16000 )
    

base_path='./data/'
dir_list = os.listdir(base_path)
print(dir_list)
for item1 in dir_list:
  work_path   = base_path+item1
  output_path = base_path+item1+'_16k'
  #print(work_path)
  #print(output_path)
  if not os.path.exists(output_path):
    print('subsampling for '+output_path)
    os.mkdir(output_path)
  file_list_subsamples=os.listdir(work_path) # [:40] for debugging
  for item2 in file_list_subsamples:
    input_file=work_path+'/'+item2
    output_file=output_path+'/'+item2
    #print(input_file)
    #print(output_file)
    tfm.build(input_file, output_file)

['noisy_testset_wav', 'clean_trainset_wav', 'clean_testset_wav', 'noisy_trainset_wav']
subsampling for ./data/noisy_testset_wav_16k
subsampling for ./data/clean_trainset_wav_16k
subsampling for ./data/clean_testset_wav_16k
subsampling for ./data/noisy_trainset_wav_16k
CPU times: user 13.8 s, sys: 45.1 s, total: 58.9 s
Wall time: 4min 41s


after subsampling, we could get  new folder name with  `_16k`. As you see,  dataset size become half of original size as below :  
```
1.1G	data/clean_trainset_wav
1.1G	data/noisy_trainset_wav
192M	data/clean_testset_wav
192M	data/noisy_testset_wav
3.7G	data
540M	data/clean_trainset_wav_16k
540M	data/noisy_trainset_wav_16k
97M	data/clean_testset_wav_16k
97M	data/noisy_testset_wav_16k
```

In [7]:
!du -h data | sort
!ls -alh $output_path| tail -n 4


1.1G	data/clean_trainset_wav
1.1G	data/noisy_trainset_wav
192M	data/clean_testset_wav
192M	data/noisy_testset_wav
3.7G	data
540M	data/clean_trainset_wav_16k
540M	data/noisy_trainset_wav_16k
97M	data/clean_testset_wav_16k
97M	data/noisy_testset_wav_16k
-rw-r--r--  1 root root  38K Oct  3 06:48 p287_421.wav
-rw-r--r--  1 root root  46K Oct  3 06:47 p287_422.wav
-rw-r--r--  1 root root  64K Oct  3 06:47 p287_423.wav
-rw-r--r--  1 root root  46K Oct  3 06:47 p287_424.wav


In [8]:
!nvidia-smi  

Wed Oct  3 06:54:26 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [10]:
'''
this is critical option. 
TF memory control allow_growth is mandatory option for COLAB
'''
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config )
print("tensorflow version : "+tf.__version__)
!nvidia-smi  | grep MiB

tensorflow version : 1.11.0
| N/A   36C    P0    68W / 149W |    115MiB / 11439MiB |      0%      Default |


## clone the SEGAN model and utilities
The origianl SEGAN implementation based python 2.7 so I've  modify few word for compatability with COLAB which use python version 3.6.3 
you could check [diff](https://github.com/yhgon/segan/commit/20fd90c04d64fa45d856963ee487e186a66db013)
```diff
92 -        for dset_i, (dset, dset_desc) in enumerate(cfg_desc.iteritems()):
92 +       for dset_i, (dset, dset_desc) in enumerate(cfg_desc.items()):
```


In [29]:
%%time
# clone yhgon's fork for segan with compatability of python 3.6.3 in colab
!git clone https://github.com/yhgon/segan.git

Cloning into 'segan'...
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 299 (delta 0), reused 0 (delta 0), pack-reused 297
Receiving objects: 100% (299/299), 787.93 KiB | 19.22 MiB/s, done.
Resolving deltas: 100% (161/161), done.
CPU times: user 21.6 ms, sys: 11.5 ms, total: 33.2 ms
Wall time: 2.06 s


## make TFrecords 
in this step, we make TFrecords files from raw wave, the default configuration is on `cfg/e2e_maker.cfg` for clean/noisy dataset location. The number of total files  are 11,572 and it takes about 25 sec to finish. 

In [14]:
%%time
#prepare tfrecords
!python ./segan/make_tfrecords.py --force-gen  --save_path './data' --cfg './segan/cfg/e2e_maker.cfg'

--------------------------------------------------
Processing wav file 11572/11572 data/clean_trainset_wav_16k/p243_386.wav          
**************************************************
Total processing and writing time: 23.500228961999937 s
CPU times: user 478 ms, sys: 132 ms, total: 610 ms
Wall time: 28.2 s


In [15]:
!ls -lah ./data/segan.tfrecords

-rw-r--r-- 1 root root 2.0G Oct  3 07:07 ./data/segan.tfrecords


I strongly recommend to backup TFrecords files. you could re-use 2GB TFrecords files the location lis below:   ./data/segan.tfrecords




In [16]:
%%time

!cp  -rf ./data/segan.tfrecords "drive/My Drive/COLAB/segan/."
!ls -alh   "drive/My Drive/COLAB/segan/"

clean_testset_wav.zip	noisy_testset_wav.zip	segan.tfrecords
clean_trainset_wav.zip	noisy_trainset_wav.zip
CPU times: user 105 ms, sys: 39 ms, total: 144 ms
Wall time: 18 s


In [31]:
#!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1404/x86_64/libcudnn5_5.1.10-1+cuda8.0_amd64.deb
!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1404/x86_64/libcudnn5-dev_5.1.10-1+cuda8.0_amd64.deb
!pip install tensorflow_gpu==0.12.1


Redirecting output to ‘wget-log.1’.


In [32]:
!ls -lah

total 72M
drwxr-xr-x  1 root root 4.0K Oct  3 07:48 .
drwxr-xr-x  1 root root 4.0K Oct  3 06:35 ..
drwxr-xr-x  4 root root 4.0K Sep 28 23:11 .config
drwxr-xr-x 10 root root 4.0K Oct  3 07:06 data
drwx------  2 root root 4.0K Oct  3 06:40 dataset-segan
drwx------  2 root root 4.0K Oct  3 06:39 drive
drwxr-xr-x  2 root root 4.0K Oct  3 07:25 dwavegan_samples
-rw-r--r--  1 root root  40M Sep 26 23:24 libcudnn5_5.1.10-1+cuda8.0_amd64.deb
-rw-r--r--  1 root root  33M Sep 26 23:24 libcudnn5-dev_5.1.10-1+cuda8.0_amd64.deb
drwxr-xr-x  2 root root 4.0K Sep 28 23:32 sample_data
drwxr-xr-x  6 root root 4.0K Oct  3 07:25 segan
drwxr-xr-x  3 root root 4.0K Oct  3 07:37 segan_naive
drwxr-xr-x  2 root root 4.0K Oct  3 07:25 segan_result_allbiased_preemph
-rw-r--r--  1 root root 1.8K Oct  3 07:44 wget-log
-rw-r--r--  1 root root 1.5K Oct  3 07:48 wget-log.1


In [33]:
!nvcc --version

/bin/bash: nvcc: command not found


In [35]:
#! dpkg -i libcudnn5_5.1.10-1+cuda8.0_amd64.deb
!dpkg -i libcudnn5-dev_5.1.10-1+cuda8.0_amd64.deb

Selecting previously unselected package libcudnn5-dev.
(Reading database ... 18918 files and directories currently installed.)
Preparing to unpack libcudnn5-dev_5.1.10-1+cuda8.0_amd64.deb ...
Unpacking libcudnn5-dev (5.1.10-1+cuda8.0) ...
Setting up libcudnn5-dev (5.1.10-1+cuda8.0) ...
update-alternatives: using /usr/include/x86_64-linux-gnu/cudnn_v5.h to provide /usr/include/cudnn.h (libcudnn) in auto mode


In [39]:
!ls /usr/lib64-nvidia

10_nvidia.json			libGLESv2.so
libcuda.so			libGLESv2.so.2
libcuda.so.1			libnvidia-cfg.so
libcuda.so.384.111		libnvidia-cfg.so.1
libcudnn.so			libnvidia-cfg.so.384.111
libcudnn.so.6			libnvidia-compiler.so
libcudnn.so.6.0.21		libnvidia-compiler.so.384.111
libEGL_nvidia.so		libnvidia-eglcore.so
libEGL_nvidia.so.0		libnvidia-eglcore.so.384.111
libEGL_nvidia.so.384.111	libnvidia-fatbinaryloader.so
libEGL.so			libnvidia-fatbinaryloader.so.384.111
libEGL.so.1			libnvidia-glsi.so
libEGL.so.384.111		libnvidia-glsi.so.384.111
libGLdispatch.so		libnvidia-ml.so
libGLdispatch.so.0		libnvidia-ml.so.1
libGLESv1_CM_nvidia.so		libnvidia-ml.so.384.111
libGLESv1_CM_nvidia.so.1	libnvidia-ptxjitcompiler.so
libGLESv1_CM_nvidia.so.384.111	libnvidia-ptxjitcompiler.so.1
libGLESv1_CM.so			libnvidia-ptxjitcompiler.so.384.111
libGLESv1_CM.so.1		libnvidia-tls.so
libGLESv2_nvidia.so		libnvidia-tls.so.384.111
libGLESv2_nvidia.so.2		libOpenGL.so
libGLESv2_nvidia.so.384.111	libOpenGL.so.0


In [0]:
!rm -r /usr/lib64-nvidia/libcudnn.so.6*

In [3]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=17.10
DISTRIB_CODENAME=artful
DISTRIB_DESCRIPTION="Ubuntu 17.10"
NAME="Ubuntu"
VERSION="17.10 (Artful Aardvark)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 17.10"
VERSION_ID="17.10"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=artful
UBUNTU_CODENAME=artful


In [7]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604_8.0.61-1_amd64.deb


Redirecting output to ‘wget-log’.


In [8]:
!ls -alh

total 32K
drwxr-xr-x 1 root root 4.0K Oct  3 10:36 .
drwxr-xr-x 1 root root 4.0K Oct  3 10:02 ..
drwxr-xr-x 4 root root 4.0K Sep 28 23:11 .config
-rw-r--r-- 1 root root 2.7K Sep 22  2017 cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
-rw-r--r-- 1 root root 1.5K Oct  3 10:16 cudasign.pub
drwxr-xr-x 2 root root 4.0K Sep 28 23:32 sample_data
-rw-r--r-- 1 root root  781 Oct  3 10:36 wget-log


Selecting previously unselected package cuda-repo-ubuntu1604.
(Reading database ... 18496 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1604_8.0.61-1_amd64.deb ...
Unpacking cuda-repo-ubuntu1604 (8.0.61-1) ...
Setting up cuda-repo-ubuntu1604 (8.0.61-1) ...

Configuration file '/etc/apt/sources.list.d/cuda.list'
 ==> File on system created by you or by a script.
 ==> File also in package provided by package maintainer.
   What would you like to do about it ?  Your options are:
    Y or I  : install the package maintainer's version
    N or O  : keep your currently-installed version
      D     : show the differences between the versions
      Z     : start a shell to examine the situation
 The default action is to keep your current version.
*** cuda.list (Y/I/N/O/D/Z) [default=N] ? Y
Installing new version of config file /etc/apt/sources.list.d/cuda.list ...

Redirecting output to ‘wget-log’.
Note: Check first if apt-key functionality is needed at all -

In [17]:
!apt-get update 

Ign:1 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease
Hit:2 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Release
Hit:3 http://security.ubuntu.com/ubuntu artful-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu artful InRelease
Hit:6 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Reading package lists... Done


In [31]:
!  apt-get install  -y --no-install-recommends cuda-core-8-0  \
  cuda-cublas-dev-8-0  cuda-cudart-dev-8-0   \
  cuda-cufft-dev-8-0   cuda-curand-dev-8-0  \
  cuda-cusolver-dev-8-0   cuda-cusparse-dev-8-0 \
  cuda-misc-headers-8-0   cuda-npp-dev-8-0 \
  cuda-nvgraph-dev-8-0 cuda-nvml-dev-8-0   \
  cuda-nvrtc-dev-8-0 


Reading package lists... Done
Building dependency tree       
Reading state information... Done
cuda-core-8-0 is already the newest version (8.0.61-1).
cuda-cublas-dev-8-0 is already the newest version (8.0.61.2-1).
cuda-cudart-dev-8-0 is already the newest version (8.0.61-1).
cuda-cufft-dev-8-0 is already the newest version (8.0.61-1).
cuda-curand-dev-8-0 is already the newest version (8.0.61-1).
cuda-cusolver-dev-8-0 is already the newest version (8.0.61-1).
cuda-cusparse-dev-8-0 is already the newest version (8.0.61-1).
cuda-misc-headers-8-0 is already the newest version (8.0.61-1).
cuda-npp-dev-8-0 is already the newest version (8.0.61-1).
cuda-nvgraph-dev-8-0 is already the newest version (8.0.61-1).
cuda-nvml-dev-8-0 is already the newest version (8.0.61-1).
cuda-nvrtc-dev-8-0 is already the newest version (8.0.61-1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [33]:
!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1404/x86_64/libcudnn5_5.1.10-1+cuda8.0_amd64.deb
!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1404/x86_64/libcudnn5-dev_5.1.10-1+cuda8.0_amd64.deb
!dpkg -i libcudnn5_5.1.10-1+cuda8.0_amd64.deb
!dpkg -i libcudnn5-dev_5.1.10-1+cuda8.0_amd64.deb



Redirecting output to ‘wget-log.4’.

Redirecting output to ‘wget-log.5’.
(Reading database ... 19909 files and directories currently installed.)
Preparing to unpack libcudnn5_5.1.10-1+cuda8.0_amd64.deb ...
Unpacking libcudnn5 (5.1.10-1+cuda8.0) over (5.1.10-1+cuda8.0) ...
Setting up libcudnn5 (5.1.10-1+cuda8.0) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
(Reading database ... 19909 files and directories currently installed.)
Preparing to unpack libcudnn5-dev_5.1.10-1+cuda8.0_amd64.deb ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
Unpacking libcudnn5-dev (5.1.10-1+cuda8.0) over (5.1.10-1+cuda8.0) ...
Setting up libcudnn5-dev (5.1.10-1+cuda8.0) ...
update-alternatives: using /usr/include/x86_64-linux-gnu/cudnn_v5.h to provide /usr/include/cudnn.h (libcudnn) in auto mode


In [32]:
!ls /usr/lib64-nvidia

10_nvidia.json			libGLESv2.so
libcuda.so			libGLESv2.so.2
libcuda.so.1			libnvidia-cfg.so
libcuda.so.384.111		libnvidia-cfg.so.1
libcudnn.so			libnvidia-cfg.so.384.111
libcudnn.so.6			libnvidia-compiler.so
libcudnn.so.6.0.21		libnvidia-compiler.so.384.111
libEGL_nvidia.so		libnvidia-eglcore.so
libEGL_nvidia.so.0		libnvidia-eglcore.so.384.111
libEGL_nvidia.so.384.111	libnvidia-fatbinaryloader.so
libEGL.so			libnvidia-fatbinaryloader.so.384.111
libEGL.so.1			libnvidia-glsi.so
libEGL.so.384.111		libnvidia-glsi.so.384.111
libGLdispatch.so		libnvidia-ml.so
libGLdispatch.so.0		libnvidia-ml.so.1
libGLESv1_CM_nvidia.so		libnvidia-ml.so.384.111
libGLESv1_CM_nvidia.so.1	libnvidia-ptxjitcompiler.so
libGLESv1_CM_nvidia.so.384.111	libnvidia-ptxjitcompiler.so.1
libGLESv1_CM.so			libnvidia-ptxjitcompiler.so.384.111
libGLESv1_CM.so.1		libnvidia-tls.so
libGLESv2_nvidia.so		libnvidia-tls.so.384.111
libGLESv2_nvidia.so.2		libOpenGL.so
libGLESv2_nvidia.so.384.111	libOpenGL.so.0


In [0]:
!rm -rf /usr/lib64-nvidia/libcu*

In [27]:
!pip install tensorflow_gpu==0.12.1

    100% |████████████████████████████████| 89.7MB 231kB/s 


In [28]:
%%time
!ls -h   "drive/My Drive/COLAB/segan/"
!cp  -rf "drive/My Drive/COLAB/segan/segan.tfrecords" .
!mkdir data
!cp -rf segan.tfrecords data/.

clean_testset_wav.zip	noisy_testset_wav.zip	segan.tfrecords
clean_trainset_wav.zip	noisy_trainset_wav.zip
CPU times: user 187 ms, sys: 46.3 ms, total: 234 ms
Wall time: 34.5 s


## train the model

11GB for batch 32


In [35]:
%%time
!CUDA_VISIBLE_DEVICES="0" \
    python ./segan/main.py \
    --epoch 80 \
    --batch_size 32 \
    --save_freq 50 \
    --init_noise_std 0.\
    --init_l1_weight 100. \
    --save_path segan_naive \
    --g_nl prelu \
    --model gan 


I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcublas.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcudnn.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcufft.so locally
I tensorflow/stream_executor/dso_loader.cc:119] Couldn't open CUDA library libcuda.so.1. LD_LIBRARY_PATH: /usr/lib64-nvidia
I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:165] hostname: 7bb4d9ae0eb6
I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:189] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:363] driver version file contents: """NVRM version: NVIDIA UNIX x86_64 Kernel Module  384.111  Tue Dec 19 23:51:45 PST 2017
GCC version:  Selected multilib: .;@m64
"""
I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:193] kernel reported version is: 384.111.0
I tensorflow/

In [0]:
!which tensorflow